In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Batch upserting Holdings

Illustrates how large payload of Holdings data that exceeds API limits can be batched.

Attributes
----------
Holdings
Batch Upsert
"""

toggle_code("Toggle Docstring")

# Upserting a Large Payload of Holdings

In this notebook, we will demonstrate how you can divide a larger holdings payload into smaller chunks. This can be useful in case you hit the payload limit in a Set Holdings or Adjust Holdings call.

## Table of Contents:
- 1. [Creating Sample Holdings Data](#1.-Creating-Sample-Holdings-Data)
- 2. [Dividing the Data](#2.-Dividing-the-Data)
- 3. [Upserting the Parts](#3.-Upserting-the-Parts)

In [13]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time

import random
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
)
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)


LUSID Environment Initialised
LUSID API Version : 0.6.10480.0


In [14]:
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)

# 1. Creating Sample Holdings Data

We will first set up some parameters, here we can alter the example of the notebook. n_instruments sets how many securities we wish to upsert holdings of. n_max_holdings will determine how large our parts can be when we later divide the total set of holdings.

In [15]:
scope = "largePayload"
portfolio_code = "largePayLoadPortfolio"
n_instruments = 5000
n_max_holdings = 1000

We also create an example portfolio to hold the data.

In [16]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:

        if "PortfolioWithIdAlreadyExists" not in str(e.body):

          print(e)

We will now create a function that will generate a pandas dataframe with n number of instruments. This number is determined above and in our case is 5000. This also determines the number of holdings we will be upserting. Our quantity and costs will be randomly generated.

In [17]:
# Create a function to generate a row
def add_row(instrument, quantity, cost, currency):
    new_row = {'client_internal': instrument,
           'quantity': quantity,
           'cost': cost,
           'total_cost': quantity*cost,
           'currency': currency}
    return new_row

# Create a dictionary with the data for the dataframe
data = {'client_internal': [],
        'quantity': [],
        'cost': [],
        'total_cost': [],
        'currency': []}

# Create the dataframe
holdings = pd.DataFrame(data)

# Create the example data

for i in range(0, n_instruments):  
    to_append = add_row(f"instrument{i}", random.randint(10, 400), round(random.uniform(5, 100), 2), "USD")
    holdings = holdings.append(to_append, ignore_index=True)

holdings.tail()

,client_internal,quantity,cost,total_cost,currency
4995,instrument4995,349.00,88.15,"30,764.35",USD
4996,instrument4996,184.00,59.35,"10,920.40",USD
4997,instrument4997,40.00,94.30,"3,772.00",USD
4998,instrument4998,86.00,36.05,"3,100.30",USD
4999,instrument4999,121.00,12.04,"1,456.84",USD


In order for the instruments to be recognised, they must be upserted to the instrument master first.

In [18]:
instrument_mapping_required = {
    "name": "client_internal",
    "currency": "currency"
}

response = load_from_data_frame(
    scope = scope,
    api_factory=api_factory, 
    data_frame=holdings, 
    identifier_mapping={"ClientInternal": "client_internal"}, 
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type='instruments',
)

succ, failed, errors = format_instruments_response(response)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,5000,0,0


Finally, we generate our list of holdings that will be upserted. In this case, it will generate 5000 holdings.

In [19]:
holding_adjustments = []

# Iterate over your holdings
for index, holding in holdings.iterrows():

    # Create a holding adjustment for this holding
    holding_adjustments.append(
        lm.AdjustHoldingRequest(
            instrument_identifiers={"Instrument/default/ClientInternal": holding["client_internal"]},
            tax_lots=[
                lm.TargetTaxLotRequest(
                    units=holding["quantity"],
                    cost=lm.CurrencyAndAmount(
                        amount=holding["total_cost"], currency=holding["currency"]
                    ),
                    portfolio_cost=holding["total_cost"],
                    price=holding["cost"]
                )
            ],
        )
    )

# 2. Dividing the Data

Now that we have our example data, we will write a function that takes in an array and divides it into smaller parts given the max amount of holdings one part may have. We set the max amount of holdings to the n_max_holdings number we determined earlier. In our case this will be 1000 and therefore will split the array of 5000 holdings into 5 arrays with 1000 holdings.

In [20]:
def split_array(input_array, n):
    
    length = len(input_array)
    
    # get the number of sub arrays
    num_subarrays = length // n
    
    subarrays = []
    for i in range(num_subarrays):
        # get the start and end indices for the subarray
        start = i * n
        end = (i + 1) * n
        # get the subarray and append it to the list of subarrays
        subarrays.append(input_array[start:end])
    
    # create an additional subarray with the remaining elements
    if length % n != 0:
        subarrays.append(input_array[num_subarrays * n:])
    
    return subarrays

In [21]:
sub_arrays = split_array(holding_adjustments, n_max_holdings)

# 3. Upserting the Parts

Now that we have an array that contains 5 smaller sub arrays with 1000 values each, we can loop through this array and upsert each of the 5 smaller subarrays. This will result in 5 batches being upserted one at a time with a lower payload.

In [22]:
effective_date = datetime.now(pytz.UTC)

counter = 1
for array in sub_arrays:
    response = transaction_portfolios_api.adjust_holdings(
        scope=scope,
        code="largePayLoadPortfolio",
        effective_at=effective_date,
        adjust_holding_request=array)
    print(f"Batch {counter} upserted.")
    counter +=1


Batch 1 upserted.
Batch 2 upserted.
Batch 3 upserted.
Batch 4 upserted.
Batch 5 upserted.


Now that we have upserted the 5 batches, we want to verify the total number of holdings uploaded. Let's call the holdings of the portfolio and return the length of the list. This should equal to 5000 as we just uploaded 5 batches of 1000 holdings. 

In [23]:
holdings = transaction_portfolios_api.get_holdings(scope=scope, code=portfolio_code)
len(holdings.values)

5000